# Exercise 1

In [ ]:
##Ejemplo

# Exercise 2

In [ ]:
def kMeans(A, k, eps, mu = None):
    #Implementacion 

In [ ]:
def DBSCAN(D, epsilon, minpts):
    #Implementacion

# Exercise 3

In [ ]:
def plotClusters(A, C, dimX, dimY, dimZ = None, ax = None):
    #Implementacion

In [ ]:
#Correr el codigo con PCA 18 veces ir plottear